# 对话代理 Conversational agent

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、逐步构建Agent](#二、逐步构建Agent)
     - [2.1 定义Tools和Function](##2.1-定义Tools和Function)
     - [2.2 加入中间步骤结果](##2.2-加入中间步骤结果)
     - [2.3 加入对话历史](##2.3-加入对话历史)
 - [三、创建对话机器人](#三、创建对话机器人)
 - [四、英文版提示](#四、英文版提示)

# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、逐步构建Agent

## 2.1 定义Tools和Function

In [1]:
# 导入tool包
from langchain.tools import tool

In [2]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入格式
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="要获取天气数据的位置的纬度") 
    longitude: float = Field(..., description="要获取天气数据的位置的经度") 

# 使用 @tool 装饰器并指定输入格式
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"根据给定的坐标位置获得温度"""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.utcnow()
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'现在温度是 {current_temperature}°C'


In [3]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"页面: {page_title}\n摘要: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "维基百科没有搜索到合适的结果"
    return "\n\n".join(summaries)

In [4]:
# 加入tools列表
tools = [get_current_temperature, search_wikipedia]

In [9]:
# from langchain.chat_models import ChatOpenAI，最新版本使用以下方式代替该语句，且调用中需要添加opai_api_key这个参数
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [10]:
# 将工具格式化为 OpenAI 函数
functions = [format_tool_to_openai_function(f) for f in tools]

# 创建 ChatOpenAI 模型，设置温度为 0，绑定生成的 OpenAI 函数列表
model = ChatOpenAI(temperature=0,openai_api_key="your_api_key").bind(functions=functions)

# 创建 ChatPromptTemplate，从消息模板中获取用户输入
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | OpenAIFunctionsAgentOutputParser()


In [11]:
# 调用
result = chain.invoke({"input": "现在圣佛朗西斯科的温度是多少?"})

In [12]:
# 查看调用的工具
result.tool

'get_current_temperature'

In [13]:
# 查看工具的输入
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [14]:
result.tool

'get_current_temperature'

In [15]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

## 2.2 加入中间步骤结果

In [16]:
# 创建 ChatPromptTemplate，从消息模板中获取用户输入
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个安全且乐于助人的助手"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # 解析和验证关键字参数中的输入数据
])

In [17]:
# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [18]:
# 调用
result1 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?",
    "agent_scratchpad": []
})

In [19]:
# 查看调用的工具
result1.tool

'get_current_temperature'

In [20]:
# 获取tool调用的结果
observation = get_current_temperature(result1.tool_input)

/Users/jingllinzhou/anaconda3/envs/extractor/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_13052/4201672864.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [21]:
# 查看结果
observation

'现在温度是 14.9°C'

In [22]:
# 查看输出类型
type(result1)

langchain_core.agents.AgentActionMessageLog

In [23]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [24]:
# 查看调用日志
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 179, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9e315c35-ee45-4de5-bf34-b81c4757910a-0')]

In [25]:
# 将Function和Tool的结果转换成openai函数格式
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 179, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9e315c35-ee45-4de5-bf34-b81c4757910a-0'),
 FunctionMessage(content='现在温度是 14.9°C', name='get_current_temperature')]

In [26]:
# 构造成Agent的输入格式，agent_scratchpad是中间步骤的结果
result2 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [27]:
# 查看Agent的输出结果
result2

AgentFinish(return_values={'output': '现在圣佛朗西斯科的温度是14.9°C。'}, log='现在圣佛朗西斯科的温度是14.9°C。')

In [29]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    # 存储中间步骤的列表
    intermediate_steps = []

    # 循环执行 agent 操作
    while True:
        # 调用处理链的 invoke 方法，传递用户输入和中间步骤的 OpenAI 函数形式
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })

        # 如果结果是 AgentFinish 类型，则结束运行并返回结果
        if isinstance(result, AgentFinish):
            return result

        # 根据结果中的工具名称选择相应的工具函数
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]

        # 运行选定的工具函数，并获取观察结果
        observation = tool.run(result.tool_input)

        # 将结果和观察结果添加到中间步骤列表
        intermediate_steps.append((result, observation))


In [30]:
from langchain.schema.runnable import RunnablePassthrough

# 创建 RunnablePassthrough，用于将 agent_scratchpad存储的中间步骤结果转换为 OpenAI 函数形式
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain


In [31]:
# 定义一个运行 agent 的函数
def run_agent(user_input):
    # 存储中间步骤的列表
    intermediate_steps = []

    # 循环执行 agent 操作
    while True:
        # 通过 agent_chain 调用 invoke 方法，传递用户输入和中间步骤的列表
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })

        # 如果结果是 AgentFinish 类型，则结束循环并返回结果
        if isinstance(result, AgentFinish):
            return result

        # 根据结果中的工具名称选择相应的工具函数
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]

        # 运行选定的工具函数，并获取观察结果
        observation = tool.run(result.tool_input)

        # 将结果和观察结果添加到中间步骤列表
        intermediate_steps.append((result, observation))


In [32]:
run_agent("圣弗朗西斯科的温度是多少？")

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_13052/4201672864.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


AgentFinish(return_values={'output': '圣弗朗西斯科现在的温度是14.9°C。'}, log='圣弗朗西斯科现在的温度是14.9°C。')

In [33]:
run_agent("什么是langchain?")

AgentFinish(return_values={'output': 'LangChain是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例主要与语言模型的用例重叠，包括文档分析和摘要、聊天机器人以及代码分析。'}, log='LangChain是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例主要与语言模型的用例重叠，包括文档分析和摘要、聊天机器人以及代码分析。')

In [34]:
run_agent("你好！")

AgentFinish(return_values={'output': '你好！有什么可以帮助你的吗？'}, log='你好！有什么可以帮助你的吗？')

In [35]:
# 使用AgentExecutor对agent进行封装
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "什么是langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


页面: LangChain
摘要: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

页面: DataStax
摘要: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.

页面: Sentence embedding
摘要: In natural language processing, a sentence embedding refers to a numeric represe

{'input': '什么是langchain?',
 'output': 'LangChain是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例主要与语言模型的用例重叠，包括文档分析和摘要、聊天机器人以及代码分析。'}

In [37]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
你好，Bob！有什么可以帮助你的吗？

> Finished chain.


{'input': '我的名字是bob', 'output': '你好，Bob！有什么可以帮助你的吗？'}

In [38]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
您的名字是{{user_name}}。

> Finished chain.


{'input': '我的名字是什么？', 'output': '您的名字是{{user_name}}。'}

## 3.3 加入对话历史

In [39]:
# 加入用户的对话历史
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [40]:
# 构造agent的处理链
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [41]:
# 加入对话记忆模块
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [42]:
# 封装AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [43]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
你好，Bob！有什么可以帮助你的吗？

> Finished chain.


{'input': '我的名字是bob',
 'chat_history': [HumanMessage(content='我的名字是bob'),
  AIMessage(content='你好，Bob！有什么可以帮助你的吗？')],
 'output': '你好，Bob！有什么可以帮助你的吗？'}

In [44]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
你的名字是Bob。有什么其他问题需要帮忙解决的吗？

> Finished chain.


{'input': '我的名字是什么？',
 'chat_history': [HumanMessage(content='我的名字是bob'),
  AIMessage(content='你好，Bob！有什么可以帮助你的吗？'),
  HumanMessage(content='我的名字是什么？'),
  AIMessage(content='你的名字是Bob。有什么其他问题需要帮忙解决的吗？')],
 'output': '你的名字是Bob。有什么其他问题需要帮忙解决的吗？'}

In [45]:
agent_executor.invoke({"input": "圣弗朗西斯科现在的温度是多少？"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`




/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_13052/4201672864.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


现在温度是 14.9°C圣弗朗西斯科现在的温度是14.9°C。有什么其他问题需要帮忙解决的吗？

> Finished chain.


{'input': '圣弗朗西斯科现在的温度是多少？',
 'chat_history': [HumanMessage(content='我的名字是bob'),
  AIMessage(content='你好，Bob！有什么可以帮助你的吗？'),
  HumanMessage(content='我的名字是什么？'),
  AIMessage(content='你的名字是Bob。有什么其他问题需要帮忙解决的吗？'),
  HumanMessage(content='圣弗朗西斯科现在的温度是多少？'),
  AIMessage(content='圣弗朗西斯科现在的温度是14.9°C。有什么其他问题需要帮忙解决的吗？')],
 'output': '圣弗朗西斯科现在的温度是14.9°C。有什么其他问题需要帮忙解决的吗？'}

# 三、 创建对话机器人

In [46]:
# 自定义的工具（自由发挥）
@tool
def create_your_own(query: str) -> str:
    """可以自定义的功能函数 """
    print(type(query))
    return query[::-1]

In [47]:
# 将之前定义的工具加入工具列表
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [50]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

# 定义 cbfs 类
class cbfs(param.Parameterized):
    
    # 初始化函数
    def __init__(self, tools, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []  # 存储 GUI 面板
        self.functions = [format_tool_to_openai_function(f) for f in tools]  # 将tools格式化为 OpenAI 函数
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)  # 创建 ChatOpenAI 模型
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")  # 创建 ConversationBufferMemory
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])  # 创建 ChatPromptTemplate
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()  # 创建处理链
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)  # 创建 AgentExecutor
    
    # 对话链函数
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

    # 清除历史记录函数
    def clr_history(self, count=0):
        self.chat_history = []
        return


/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_13052/3798240064.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


In [ ]:
# 创建 cbfs 对象，传递工具列表
cb = cbfs(tools)

# 创建文本输入框组件
inp = pn.widgets.TextInput(placeholder='请输入文本...')

# 将 convchain 方法与输入框进行绑定，形成对话链
conversation = pn.bind(cb.convchain, inp) 

# 创建第一个选项卡组件
tab1 = pn.Column(
    pn.Row(inp),  # 显示文本输入框
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),  # 显示对话链面板
    pn.layout.Divider(),
)

# 创建仪表板，包含标题和选项卡
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# 对话机器人')),  # 显示标题
    pn.Tabs(('发送', tab1))  # 创建选项卡
)

# 显示仪表板
dashboard


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='请输入文本...')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [3] Divider()

聊天机器人效果展示如下：

![](../../figures/chat_bot.png)

# 四、英文版提示

总结一下构建对话机器人的流程：

定义tools和functions --> 构建模型 --> 加入中间步骤结果 --> 加入记忆机制 --> 交互界面  

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
# 构造agent的处理链
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
# 加入对话记忆模块
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [ ]:
# 封装AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'input': 'what is my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='what is my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?')],
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


现在温度是 9.5°CThe current temperature in San Francisco is 9.5°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='what is my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 9.5°C.')],
 'output': 'The current temperature in San Francisco is 9.5°C.'}